# Basic usage

*`inferno`* is designed to maximize interoperability between `pytorch` and `sklearn`. The aim is to keep 99% of the flexibility of `pytorch` while being able to leverage most features of `sklearn`. Below, we show the basic usage of `inferno` and how it can be combined with `sklearn`.

### Table of contents

* [Definition of the pytorch module](#Definition-of-the-pytorch-module)
* [Training a model and predicting](#Training-a-model-and-predicting)
* [Saving and loading a model](#Saving-and-loading-a-model)
* [Usage with an sklearn Pipeline](#Usage-with-an-sklearn-Pipeline)
* [Callbacks](#Callbacks)
* [Grid search](#Usage-with-sklearn-GridSearchCV)

## Toy dataset

This is a toy binary classification task

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
torch.manual_seed(0)

In [3]:
import numpy as np
from sklearn.datasets import make_classification

In [4]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)

In [5]:
X.shape, y.shape, y.mean()

((1000, 20), (1000,), 0.5)

## Definition of the `pytorch module`

We define a vanilla neural network with two hidden layers. The output layer should have 2 output units since there are two classes. In addition, it should have a softmax nonlinearity, because later, when calling `predict_proba`, the output from the `forward` call will be used.

In [6]:
class MyModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(MyModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X))
        return X

## Training a model and predicting

We use `NeuralNetClassifier` because we're dealing with a classifcation task. The first argument should be the `pytorch module`. As additional arguments, we pass the number of epochs and the learning rate (`lr`), but those are optional.

In [7]:
from inferno.net import NeuralNetClassifier

In [8]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=20,
    lr=0.1,
)

As in `sklearn`, we call `fit` passing the input data `X` and the targets `y`. By default, `NeuralNetClassifier` makes a `StratifiedKFold` split on the data (80/20) to track the validation loss. This is shown, as well as the train loss and the accuracy on the validation set.

In [9]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7111       0.5100        0.6894  0.0651
      2        0.6928       0.5500        0.6803  0.0680
      3        0.6833       0.5650        0.6741  0.0639
      4        0.6763       0.5850        0.6674  0.0646
      5        0.6727       0.6450        0.6616  0.0651
      6        0.6606       0.6600        0.6536  0.0721
      7        0.6560       0.6600        0.6443  0.0662
      8        0.6427       0.6650        0.6354  0.0645
      9        0.6300       0.6800        0.6264  0.0767
     10        0.6289       0.6800        0.6189  0.0823
     11        0.6241       0.7150        0.6114  0.0695
     12        0.6132       0.7150        0.6017  0.0776
     13        0.5950       0.7350        0.5902  0.0707
     14        0.5914       0.7200        0.5831  0.0751
     15        0.5784       0.7300        0.5733  0.0703
     16        0.5816       0.7

Also, as in `sklearn`, you may call `predict` or `predict_proba` on the fitted model.

In [10]:
y_pred = net.predict(X[:5])
y_pred

array([1, 0, 0, 0, 0])

In [11]:
y_proba = net.predict_proba(X[:5])
y_proba

array([[ 0.33409795,  0.66590208],
       [ 0.6590634 ,  0.34093663],
       [ 0.70409262,  0.29590738],
       [ 0.70345551,  0.29654452],
       [ 0.65079045,  0.34920955]], dtype=float32)

## Saving and loading a model

Save and load either the whole model by using pickle or just the learned model parameters by calling `save_params` and `load_params`.

### The whole model

In [12]:
import pickle

In [13]:
file_name = '/tmp/mymodel.pkl'

In [14]:
with open(file_name, 'wb') as f:
    pickle.dump(net, f)

In [15]:
with open(file_name, 'rb') as f:
    new_net = pickle.load(f)

### Only the parameters

This only saves and loads the proper `module` parameters, meaning that hyperparameters such as `lr` and `max_epochs` are not saved. Therefore, to load the model, we have to re-initialize it beforehand.

In [16]:
net.save_params(file_name)  # a file handler also works

In [17]:
# first initialize the model
new_net = NeuralNetClassifier(
    MyModule,
    max_epochs=20,
    lr=0.1,
).initialize()

In [18]:
new_net.load_params(file_name)

## Usage with an `sklearn Pipeline`

It is possible to put the `NeuralNetClassifier` inside an `sklearn Pipeline`, as you would with any `sklearn` classifier.

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [20]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

In [21]:
pipe.fit(X, y)

Re-initializing module!
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7102       0.5050        0.6991  0.0634
      2        0.6971       0.5100        0.6940  0.0656
      3        0.6885       0.5350        0.6896  0.0770
      4        0.6878       0.5450        0.6863  0.0742
      5        0.6821       0.5750        0.6833  0.0662
      6        0.6845       0.5850        0.6802  0.0744
      7        0.6751       0.6450        0.6760  0.0741
      8        0.6716       0.6600        0.6719  0.0738
      9        0.6676       0.6700        0.6669  0.0676
     10        0.6575       0.6800        0.6591  0.0704
     11        0.6510       0.6900        0.6514  0.0677
     12        0.6497       0.7100        0.6439  0.0755
     13        0.6370       0.7100        0.6335  0.0714
     14        0.6274       0.7100        0.6241  0.0709
     15        0.6306       0.7200        0.6149  0.0683
     16

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('net', <inferno.net.NeuralNetClassifier object at 0x7f5ae6a5a438>)])

In [22]:
y_proba = pipe.predict_proba(X[:5])
y_proba

array([[ 0.38140488,  0.61859512],
       [ 0.67595905,  0.32404098],
       [ 0.58000326,  0.41999677],
       [ 0.65313405,  0.34686592],
       [ 0.65428513,  0.34571487]], dtype=float32)

To save the whole pipeline, including the pytorch module, use `pickle`.

## Callbacks

Adding a new callback to the model is straightforward. Below we show how to add a new score to the validation set.

### Add area under the ROC (AUC) score

In [23]:
from inferno.callbacks import Scoring
from inferno.utils import to_numpy

There is a scoring callback in inferno, `Scoring`, which we use for this. We need to specify a `name` of the score, as well as which score to calculate. Here we just pass a string, `'roc_auc_score'`, as score. For a list of all existing scores, look [here](http://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics). We could also pass a function with the signature `func(model, X, y) -> score`, or `None`, in which case the `score` method of the model is used. Note that this is exactly the same behavior as in `sklearn`.

In [24]:
auc = Scoring(
    name='AUC',
    scoring='roc_auc_score',
    lower_is_better=False,
    pred_extractor=lambda y_proba: to_numpy(y_proba)[:, 1],
)

Furthermore, we should tell the callback that higher scores are better (to get the correct colors printed below), and how to extract the data from the prediction. The latter must be specified because sometimes we need the class predictions, sometimes the class probabilities, and sometimes, as in this example, only probability of the `1` class. Moreover, we must convert the data from `torch` tensors to `numpy` arrays (using the helper function `to_numpy`).

In [25]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=20,
    lr=0.1,
    callbacks=[auc],
)

Finally, we pass the scoring callback to the `callbacks` parameter as a list and then call `fit`. Notice that we get the printed scores and color highlighting for free.

In [26]:
net.fit(X, y)

  epoch     AUC    train_loss    valid_acc    valid_loss     dur
-------  ------  ------------  -----------  ------------  ------
      1  0.6722        0.6730       0.6100        0.6730  0.0563
      2  0.6990        0.6469       0.6650        0.6621  0.0727
      3  0.7084        0.6423       0.6500        0.6548  0.0701
      4  0.7171        0.6257       0.6700        0.6482  0.1079
      5  0.7255        0.6308       0.6750        0.6402  0.0709
      6  0.7358        0.6043       0.6650        0.6330  0.0642
      7  0.7402        0.5999       0.6950        0.6277  0.0748
      8  0.7443        0.5935       0.7100        0.6238  0.0747
      9  0.7527        0.5866       0.7000        0.6095  0.0774
     10  0.7607        0.5904       0.7100        0.6039  0.0707
     11  0.7595        0.5699       0.7250        0.6007  0.0678
     12  0.7579        0.5636       0.7100        0.5992  0.0676
     13  0.7656        0.5634       0.7200        0.5903  0.0741
     14  0.7715        0.

### Writing your own callbacks

Writing your own callbacks is also straightforward. Just remember these rules:
* They should inherit from `inferno.callbacks.Callback`.
* They should implement at least one of the `on_`-methods provided by the parent class (e.g. `on_batch_begin` or `on_epoch_end`).
* As argument, the `on_`-methods first get the `NeuralNet` instance, and, where appropriate, the local data (e.g. the data from the current batch). The method should also have `**kwargs` in the signature for potentially unused arguments.

Here is an example of a callback that saves the model if the validation loss has improved.

In [27]:
from inferno.callbacks import Callback


class Checkpoint(Callback):
    def __init__(self, file_name):
        self.file_name = file_name

    def on_epoch_end(self, net, **kwargs):
        # check if valid accuracy of most recent epoch is the best so far
        if net.history[-1, 'valid_acc_best']:
            print("Save model to {}.".format(self.file_name))
            net.save_params(self.file_name)

In [28]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    callbacks=[auc, Checkpoint(file_name)],
)

In [29]:
net.fit(X, y)

Save model to /tmp/mymodel.pkl.
  epoch     AUC    train_loss    valid_acc    valid_loss     dur
-------  ------  ------------  -----------  ------------  ------
      1  0.5598        0.6962       0.5500        0.6894  0.0694
Save model to /tmp/mymodel.pkl.
      2  0.5648        0.6905       0.5550        0.6889  0.0699
Save model to /tmp/mymodel.pkl.
      3  0.5716        0.6910       0.5600        0.6883  0.0692
Save model to /tmp/mymodel.pkl.
      4  0.5773        0.6904       0.5650        0.6878  0.0726
      5  0.5810        0.6920       0.5650        0.6873  0.0684
Save model to /tmp/mymodel.pkl.
      6  0.5880        0.6889       0.5700        0.6867  0.0684
      7  0.5904        0.6923       0.5600        0.6862  0.0665
Save model to /tmp/mymodel.pkl.
      8  0.5912        0.6838       0.5750        0.6858  0.0709
Save model to /tmp/mymodel.pkl.
      9  0.5957        0.6881       0.5800        0.6853  0.0760
     10  0.6012        0.6891       0.5800        0.6849  0.0

## Usage with sklearn `GridSearchCV`

### Special prefixes

The `NeuralNet` class allows to directly access parameters of the `pytorch module` by using the `module__` prefix. So e.g. if you defined the `module` to have a `num_units` parameter, you can set it via the `module__num_units` argument. This is exactly the same logic that allows to access estimator parameters in `sklearn Pipeline`s and `FeatureUnion`s.

This feature is useful in several ways. For one, it allows to set those parameters in the model definition. Furthermore, it allows you to set parameters in an `sklearn GridSearchCV` as shown below.

In addition to the parameters prefixed by `module__`, you may access a couple of other attributes, such as those of the optimizer by using the `optim__` prefix (again, see below). All those special prefixes are stored in the `prefixes_` attribute:

In [30]:
print(', '.join(net.prefixes_))

module, iterator_train, iterator_test, optim, criterion, callbacks


### Performing a grid search

Below we show how to perform a grid search over the learning rate (`lr`), the module's number of hidden units (`module__num_units`), the module's dropout rate (`module__dropout`), and whether the SGD optimizer should use Nesterov momentum or not (`optim__nesterov`).

In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=20,
    lr=0.1,
    verbose=0,
    optim__momentum=0.9,
)

In [33]:
params = {
    'lr': [0.05, 0.1],
    'module__num_units': [10, 20],
    'module__dropout': [0, 0.5],
    'optim__nesterov': [False, True],
}

In [34]:
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy', verbose=2)

In [35]:
gs.fit(X, y)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=False 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=False, total=   1.0s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=False 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV]  lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=False, total=   1.0s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=False 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=False, total=   1.0s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=True 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=True, total=   1.0s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=True 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=True, total=   1.0s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=True 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optim__nesterov=True, total=   1.0s
[CV] lr=0.05, module__dropout=0, module__num_units=20, optim__nesterov=False 
[CV]  lr=0.05, module__dropout=0, module__num_units=20, optim__nesterov=False, total=   1.0s
[CV] lr=0.05, module__dropout=0, module__num_units=20, opt

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:   48.9s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=<inferno.net.NeuralNetClassifier object at 0x7f5ae4e01ac8>,
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'lr': [0.05, 0.1], 'module__num_units': [10, 20], 'module__dropout': [0, 0.5], 'optim__nesterov': [False, True]},
       pre_dispatch='2*n_jobs', refit=False, return_train_score=True,
       scoring='accuracy', verbose=2)

In [36]:
print(gs.best_score_, gs.best_params_)

0.856 {'lr': 0.1, 'module__dropout': 0, 'module__num_units': 20, 'optim__nesterov': True}


Of course, we could further nest the `NeuralNetClassifier` within an `sklearn Pipeline`, in which case we just prefix the parameter by the name of the net (e.g. `net__module__num_units`).